In [1]:
%matplotlib inline
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups

from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import string

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB



In [2]:
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [3]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [4]:
count_vect.vocabulary_.get(u'algorithm')

4690

In [5]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

In [6]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

In [7]:
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [8]:
docs_new = ['God is love', 'OpenGL on the GPU is fast','Act now for instant money savings','win a free ipad today']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics
'Act now for instant money savings' => soc.religion.christian
'win a free ipad today' => soc.religion.christian


In [9]:
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])

In [10]:
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

In [11]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target) 

0.83488681757656458

In [12]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)),])
_ = text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)            


0.9127829560585885

In [13]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.90      0.95      0.93       398

           avg / total       0.92      0.91      0.91      1502



In [14]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[258,  11,  15,  35],
       [  4, 379,   3,   3],
       [  5,  33, 355,   3],
       [  5,  10,   4, 379]])

In [15]:
from sklearn.grid_search import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf__alpha': (1e-2, 1e-3),}

In [16]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [17]:
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

In [18]:
twenty_train.target_names[gs_clf.predict(['God is love'])]

/Users/Grant/Documents/2015_Fall/machine_learning/jupyter/lib/python3.4/site-packages/ipykernel/__main__.py:1: DeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  if __name__ == '__main__':


'soc.religion.christian'

In [19]:
best_parameters, score, _ = max(gs_clf.grid_scores_, key=lambda x: x[1])
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

score     

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)


0.90000000000000002

# |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||




# Exercise 1

In [56]:
# """Build a language detector model

# The goal of this exercise is to train a linear classifier on text features
# that represent sequences of up to 3 consecutive characters so as to be
# recognize natural languages by using the frequencies of short character
# sequences as 'fingerprints'.

# """
# Author: Olivier Grisel <olivier.grisel@ensta.org>
# License: Simplified BSD

import sys

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_files
from sklearn.cross_validation import train_test_split
from sklearn import metrics


In [57]:

# The training data folder must be passed as first argument
#languages_data_folder = '../scikit-learn/doc/tutorial/text_analytics/data/languages/'
languages_data_folder = 'language_data/paragraphs'
dataset = load_files(languages_data_folder)


In [58]:
# Split the dataset in training and test set:
docs_train, docs_test, y_train, y_test = train_test_split(
    dataset.data, dataset.target, test_size=0.5)


In [59]:

# TASK: Build a an vectorizer that splits strings 
# into sequence of 1 to 3 characters instead of word tokens

count_vect = CountVectorizer(ngram_range=(1,3))
X_train_counts = count_vect.fit_transform(docs_train)
X_train_counts.shape

(441, 69779)

In [60]:

# TASK: Build a vectorizer / classifier pipeline using the previous analyzer
# the pipeline instance should stored in a variable named clf

clf = Pipeline([('vect', count_vect),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])

clf = clf.fit(docs_train, y_train)



In [61]:
y_predicted = clf.predict(docs_test)
np.mean(y_predicted == y_test) 

0.75791855203619907

In [62]:
print(metrics.classification_report(y_test, y_predicted,
                                    target_names=dataset.target_names))

             precision    recall  f1-score   support

         ar       0.00      0.00      0.00        13
         de       0.98      0.96      0.97        45
         en       0.38      1.00      0.55        62
         es       0.95      1.00      0.98        62
         fr       0.98      1.00      0.99        52
         it       1.00      0.91      0.95        43
         ja       0.00      0.00      0.00        34
         nl       0.00      0.00      0.00        23
         pl       0.00      0.00      0.00        23
         pt       1.00      0.96      0.98        53
         ru       1.00      0.81      0.90        32

avg / total       0.69      0.76      0.70       442



/Users/Grant/Documents/2015_Fall/machine_learning/jupyter/lib/python3.4/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [63]:
# TASK: Fit the pipeline on the training set

text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])

text_clf = text_clf.fit(docs_train, y_train)


In [64]:
# TASK: Predict the outcome on the testing set in a variable named y_predicted

y_predicted = text_clf.predict(docs_test)
np.mean(y_predicted == y_test) 


0.77828054298642535

In [65]:
# Print the classification report
print(metrics.classification_report(y_test, y_predicted,
                                    target_names=dataset.target_names))


             precision    recall  f1-score   support

         ar       0.00      0.00      0.00        13
         de       0.93      0.96      0.95        45
         en       0.43      1.00      0.60        62
         es       0.93      1.00      0.96        62
         fr       0.96      1.00      0.98        52
         it       1.00      0.98      0.99        43
         ja       1.00      0.09      0.16        34
         nl       0.00      0.00      0.00        23
         pl       1.00      0.04      0.08        23
         pt       0.91      0.96      0.94        53
         ru       1.00      0.88      0.93        32

avg / total       0.81      0.78      0.72       442



/Users/Grant/Documents/2015_Fall/machine_learning/jupyter/lib/python3.4/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [66]:

# Plot the confusion matrix
cm = metrics.confusion_matrix(y_test, y_predicted)
print(cm)



[[ 0  0 12  1  0  0  0  0  0  0  0]
 [ 0 43  1  0  0  0  0  0  0  1  0]
 [ 0  0 62  0  0  0  0  0  0  0  0]
 [ 0  0  0 62  0  0  0  0  0  0  0]
 [ 0  0  0  0 52  0  0  0  0  0  0]
 [ 0  0  1  0  0 42  0  0  0  0  0]
 [ 0  0 29  1  1  0  3  0  0  0  0]
 [ 0  1 18  3  1  0  0  0  0  0  0]
 [ 0  1 17  0  0  0  0  0  1  4  0]
 [ 0  1  1  0  0  0  0  0  0 51  0]
 [ 0  0  4  0  0  0  0  0  0  0 28]]


In [67]:

#import pylab as pl
#pl.matshow(cm, cmap=pl.cm.jet)
#pl.show()

# Predict the result on some short new sentences:
sentences = [
    u'This is a language detection test.',
    u'Ceci est un test de d\xe9tection de la langue.',
    u'Dies ist ein Test, um die Sprache zu erkennen.',
]
predicted = clf.predict(sentences)

for s, p in zip(sentences, predicted):
    print(u'The language of "%s" is "%s"' % (s, dataset.target_names[p]))



The language of "This is a language detection test." is "en"
The language of "Ceci est un test de détection de la langue." is "fr"
The language of "Dies ist ein Test, um die Sprache zu erkennen." is "de"
